In [7]:
import sys
sys.path.append("..")
import random
from sklearn.model_selection import train_test_split
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation

from movingpose.estimator import neighbors
from movingpose.estimator import classifiers

from movingpose.preprocessing import moving_pose
from movingpose.preprocessing import kinect_skeleton_data

## Pickle multiview action data

In [8]:
kinect_skeleton_data.pickle_dir("../pickle/multiview.p", "../ext/dataset/multiview_action/")

Pickling skeleton data...
Saving data...
Saved data to file: ../pickle/multiview.p
Program took 14.822474002838135 to run!


## Load pickled multiview action data

In [9]:
raw_data_dict = kinect_skeleton_data.load_pickle("../pickle/multiview.p")

## Format multiview action data

In [10]:
X, labels = moving_pose.format_skeleton_data_dict(raw_data_dict)

## Visualize formatted skeleton data

In [14]:
# initialize joint colors of skeleton
colors = ['blue' for i in range(20)]
colors[0] = 'red'  # hip is red
colors[7] = 'green'  # hands and feet are green
colors[11] = 'green'
colors[15] = 'green'
colors[19] = 'green'
colors[3] = 'black' # head is black

action_data = X[random.randrange(0, len(X))]

print(action_data)

# get array of all frames without joint number or frame number
all_frames = [np.array(i)[:,2:] for i in action_data]
max_frame = len(all_frames)

def update_graph(frame_data):
    xs, ys, zs = frame_data[:,0], frame_data[:,1], frame_data[:,2]
    graph._offsets3d = (xs, ys, zs)
#     p1 = [xs[2], ys[2], zs[2]]
#     p2 = [xs[4], ys[4], zs[4]]
#     temp = plt.plot(p1, p2)
    return graph

fig = plt.figure()
#fig.set_size_inches(9.5, 9.5)
ax = fig.add_subplot(111, projection='3d')
ax.view_init(-75,  90)
points = all_frames[0]
graph = ax.scatter(points[:,0], points[:,1], points[:,2], color=colors)

#xs, ys, zs = all_frames[0][:,0], all_frames[0][:,1], all_frames[0][:,2] ; print(xs)

ani = matplotlib.animation.FuncAnimation(fig, update_graph, all_frames, interval=90)


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2], [-0.005908022270055418, 0.05021818929547123, 0.04923351891712849, 0.0, 0.0, 0.0, -0.0010431013465507632, 0.00042571133980251474, -0.0005756536008851287, 2], [-0.026490540396846472, 0.3661598425417136, 0.05540827435516603, -0.0010217701622122696, -0.00010840796462940894, 0.0020558995386333254, -0.00110821800089414, 0.00039271811344909757, 0.0004692950746369995, 2], [-0.024480651694588844, 0.5701635458208627, 0.042343997790491104, -2.6005049226871663e-05, -5.4860048774307835e-05, 0.0031223309414033715, -0.00010236143570410694, 0.0004322714871549671, 0.0014730791838412916, 2], [-0.1898167162698159, 0.2565299436680766, 0.029678808292985765, -0.0015304903452831864, 0.0006722791457479116, 0.001975758687875081, -0.0017663261370309824, 0.0014437889862536357, 0.0001880136102059768, 2], [-0.21013352266884042, 0.03732229567860157, -0.06335077890254773, -0.0004045099273903008, 0.0013499389423551889, 0.00013170957431912766, -0.001602378569370233, 0

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## Create train/test split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, random_state=42)

sup


## Train Action Classifier with normalized training data

In [7]:
best_score = sys.maxsize
best_action_classifier = None
for n_neighbors in range(5, 15, 5):
    for n_training_neighbors in range(10, 40, 10):
        for alpha in np.arange(0.2, 0.8, 0.1):
            for beta in np.arange(0.1, 0.5, 0.1):
                for kappa in range(3, 10, 1):
                    nearest_descriptor_estimator = neighbors.NearestDescriptors(
                        n_neighbors=n_neighbors,
                        n_training_neighbors=n_training_neighbors,
                        alpha=alpha,
                        beta=beta,
                        kappa=kappa
                    )
                    for theta in np.arange(0.2, 0.8, 0.05):
                        for n in range(5, 15, 1):
                            action_classifier = classifiers.ActionClassifier(
                                nearest_descriptor_estimator=nearest_descriptor_estimator,
                                theta=theta,
                                n=n
                            )
                            action_classifier.fit(X_train, y_train)
                            y_pred = action_classifier.predict(X_test)

                            diff = 0
                            for pred, actual in zip(y_pred, y_test):
                                diff += abs(pred - actual)

                            if diff < best_score:
                                best_score = diff
                                best_action_classifier = action_classifier

KeyboardInterrupt: 

## Print best action classifier and its score

In [ ]:
print(f"The best action classifier was {best_action_classifier}")
print("\n--------------\n")
print(f"Its score was: {best_score}")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Predict live data

In [ ]:
#TODO add live predictions when mehmet publishes the UI code

# DAB

In [ ]:
print("dab")
